# Link based ranking

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, use:

In [15]:
from __future__ import print_function, division
import numpy as np
from numpy import linalg as LA

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


## Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

In [63]:
L1 = np.matrix([
    [0,0,0], 
    [1,0,0], 
    [0,0,0]
])

L2 = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])

L3 = np.matrix([
    [0,1,1], 
    [1,0,0], 
    [1,0,0]
])

L4 = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,0,0]
])

**Hints:**
We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [44]:
# Implement your code here
def pagerank_eigen(L):
#   Construct transition probability matrix from L
    R = L/L.sum(axis=0)
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = LA.eig(R)
    print(eigenvalues.argmax())
#     Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:, np.argmax(np.absolute(eigenvalues))]
    return (R,p)

In [55]:
# Test with the question, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

1
L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]
p=[[0.57735027]
 [0.57735027]
 [0.57735027]]


In [50]:
# Example from LE_06 slide 14
L = np.matrix([
    [0,0,1], 
    [1,0,0], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
M = np.multiply(R,p)
print("L={0}\nR={1}\np={2}\nM={2}".format(L,R,p,M))

0
L=[[0 0 1]
 [1 0 0]
 [1 1 0]]
R=[[0.  0.  1. ]
 [0.5 0.  0. ]
 [0.5 1.  0. ]]
p=[[-0.66666667+0.j]
 [-0.33333333+0.j]
 [-0.66666667+0.j]]
M=[[-0.66666667+0.j]
 [-0.33333333+0.j]
 [-0.66666667+0.j]]


## Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [61]:
def pagerank_iterative(L):
    R = L/L.sum(axis=0)
    N = R.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.dot(q * R,p_prev)
        p = p + (1-q) / N * e
        delta = np.linalg.norm(p-p_prev,1)
        i += 1

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

In [62]:
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
pagerank_iterative(L)

Converged after 52 iterations. Ranking vector: p=[[0.33611637]
 [0.33611637]
 [0.33611637]]


(matrix([[0. , 0.5, 0.5],
         [0.5, 0. , 0.5],
         [0.5, 0.5, 0. ]]),
 matrix([[0.33611637],
         [0.33611637],
         [0.33611637]]))

### Test with the dataset


In [ ]:
# Construct link matrix from file
f = open("ca-GrQc.txt")
L = ...

In [ ]:
# Run PageRank
R, p = pagerank_iterative(L)
print("Ranking vector: p={0}".format(p[:,0]))

## Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

(1)
- PageRank ranks node 2 higher than node 1 and 3
- HITS authority ranks 3 higher than 1 and 2
- HITS hub ranks 1 higher than 2 and 3

(2)
- PageRank has an implicit "Internet Democracy" – each website has a total of one vote
- It uses a Markov model for web surfing, instead of HITS mutual reinforcement

(3)
- Rings
- Cliques

## Question 4 - Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes. (A non-zero element $A_{ij}$ indicates an edge from $i$ to $j$.)

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [79]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]
    x0, y0 = 1 / (N*N) * np.ones(N), 1 / (N*N) * np.ones(N) 
    xprev, yprev = x0, y0
    
     # For advanced exercise: define a convergence condition instead of k iterations
    delta1 = delta2 = 1
    epsilon = 0.001 # We can strictly check for convergence rate of HITS algorithm
    l = 0
    # while l < k and delta1 > epsilon and delta2 > epsilon:
   
    for l in range(0,k):
        y = np.matmul(A, xprev)
        x = np.matmul(np.transpose(A), y) 
        x = x / np.linalg.norm(x,2)
        y = y / np.linalg.norm(y,2)
        xprev = x
        yprev = y
        # l += 1
    
    print("Ran a total of {0} iterations".format(l))
    #print("Ran a total of {0} iterations with the convergence rate delta1, delta2={1},{2}".format(l, delta1, delta2))
    return xprev, yprev

In [80]:
# Task a)
A=np.array([
    [0, 1, 1, 1], 
    [0, 0, 1, 1], 
    [1, 0, 0, 1],
    [0, 0, 0, 1],
])

x, y = hits_iterative(A, 5)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 4 iterations
Result using iterative method:
 Authoriy vector x=[0.16847843 0.27255947 0.49799459 0.80580875]
 Hub vector y=[0.65546933 0.54214151 0.40516824 0.33508393]


**Details:**
+ Initialization: 
  
  $x_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
  $y_0 = \frac{1}{4^2}(1,1,1,1) = ( 0.0625,  0.0625,  0.0625,  0.0625)$
  
+ $k=1$:
  
  $x_1 = \frac{A^t y_0}{||A^t y_0||} = (0.21320072,  0.21320072,  0.42640143,  0.85280287)$
  
  $y_1 = \frac{A x_0}{|| A x_0 ||} = (0.70710678,  0.47140452,  0.47140452,  0.23570226)$
  
+ ...:
  
+ $k=6$:
  
   $x_6 = \frac{A^t y_5}{||A^t y_5||} = (0.16887796,  0.27257494,  0.49774555, 0.80587375)$
  
  $y_6 = \frac{A x_5}{||A x_5||} = (0.65357971,  0.54153747,  0.40815386,  0.33612671)$
  

**Conclusion:**
+ Best authority node: $n_4$. Best hub node: $n_1$.
 
**Check with the theoretical result (convergence condition):**
  
+ $x^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A^t A$: $(0.16845787,  0.27257056,  0.49801119,  0.80579904)$
  
+ $y^*$ is the principal eigenvector (i.e. with largest eigenvalue) of $A A^t$: $(0.65549599,  0.54215478,  0.4051188,   0.33507008)$